# Ensemble Attack Implementation

TJ Kim, 11.16.21

#### Summary:
- Build upon the "Transferer" class that performs adversarial perturbations based on gradients of one model and sends adversarial examples to other members of ensemble
- Ensemble Cascade Adversarial Attack using multiple members of ensembles as adversarial

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


#### Import Libraries

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

#### Build New Ensemble Transferer Subclass

In [ ]:
class Ensemble_Transferer(Transferer): 
    def __init__(self, filename:str, config_name = None):
        super(Ensemble_Transferer, self).__init__(models_list, dataloader)
        
        self.models_list = models_list
        self.dataloader = dataloader 
        
        # Matrix to Record Performance (Old Metrics)
        self.orig_acc_transfers = {} # Benign point accuracy
        self.orig_similarities = {} # Benign point same output between adversary and victim
        self.orig_target_hit = {} # How many times target was hit (without evasion)
        self.adv_acc_transfers = {} # Accuracy on adversarial (how many are still classified right)
        self.adv_similarities = {} # similarity in adv output given evasion at adversary
        self.adv_target_hit = {} # How many times target was hit (due to evasion)
        
        # Matrix to Record Performance (New Metrics - theoretical)
        self.metric_variance = None # Single value
        self.metric_alignment = {} # Dict - key is victim NN id
        self.metric_ingrad = {} # Dict - key is victim NN id
        
        self.metric_alignment_robust = {} # Dict - key is victim NN id
        self.metric_ingrad_robust = {} # Dict - key is victim NN id
        
        self.metric_alignment_adv = {} # Dict - key is victim NN id
        self.metric_ingrad_adv = {} # Dict - key is victim NN id
        
        # Attack Params
        self.ifsgm_params = IFSGM_Params()
        self.ifsgm_params.x_val_min = torch.min(dataloader.x_data).item()
        self.ifsgm_params.x_val_max = torch.max(dataloader.x_data).item() 
        
        # Other Params
        self.advNN_idx = None # list
        self.advNN = None # dict of personalized_NN
        self.victim_idxs = None # List of ints
        self.victims = None # dict of pytorch nn
        self.atk_order = None
        
        # Recorded Data Points
        self.x_orig = None
        self.y_orig = None
        self.y_true = None
        self.x_adv = None
        self.y_adv = None
        
    def generate_advNN(self, client_idx):
        """
        Select specific client to load neural network to 
        Load the data for that client
        Lod the weights for that client
        This is the client that will generate perturbations
        """        
        # Import the loader for this dataset only
        
        self.advNN_idx = client_idx # List
        self.advNN = {} # Dict of NN
        
        for i in client_idx:
            self.advNN[i] = copy.deepcopy(Adv_NN(self.models_list[i], self.dataloader))
        
        return
    
    def choose_attack_sequence(self):
        """
        Given the number of iterations of attack in params, and number of adv nn
        choose a list of len~iter of which idx to attack
        must run generate_advNN prior 
        """
    
        num_iters = self.ifsgm_params.iteration
        atk_order = []
        
        for t in range(num_iters):
            idx = t%len(self.advNN_idx)
            atk_order += [self.adv_NN_idx[idx]]
            
        self.atk_order = atk_order
        
    
    def generate_xadv(self, atk_type = "IFSGM", mode='test'):
        """
        Generate perturbed images
        atk_type - "IFSGM" or "CW"
        """
        
        
        
        if (atk_type == "IFSGM") or (atk_type == "ifsgm"): 
            self.advNN.i_fgsm(self.ifsgm_params, mode=mode)
        else:
            print("Attak type unidentified -- Running IFSGM")
            self.advNN.i_fgsm(self.ifsgm_params, mode=mode)
        
        # Record relevant tensors
        self.x_orig = self.advNN.x_orig
        self.y_orig = self.advNN.output_orig
        self.y_true = self.advNN.y_orig
        self.x_adv = self.advNN.x_adv
        self.y_adv = self.advNN.output_adv
    
    def generate_victims(self, client_idxs):
        """
        Load the pre-trained other clients in the system
        """
        
        self.victim_idxs = client_idxs
        self.victims = {}
    
        for i in self.victim_idxs:
            self.victims[i] = copy.deepcopy(Personalized_NN(self.models_list[i]))